# Michał Matak - SVM do mapowania obiektu



### Importowanie bibliotek
* numpy - operacje na macierzach 
* scipy - optymalizacja parametryczna
* matplotlib - tworzenie wykresów

In [1]:
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import NonlinearConstraint
from get_dictionary import get_dictionary
import matplotlib.pyplot as plt
import csv
import random

In [14]:
directory = "series3"

### Funkcje przekształcające strukturę zbioru danych

Zakodowanie pomiarów w macierz:

In [12]:
data_dictionary = {
    "13.0_10.0":1,
    "14.0_10.0":2,
    "15.0_10.0":3,
    "15.0_11.0":4,
    "15.0_12.0":5,
    "20.0_5.0":6
}

In [13]:
data_dictionary = {
    "14.0_10.0":1,
    "15.0_10.0":2,
    "15.0_11.0":3,
}

In [15]:
data_dictionary = get_dictionary(directory)

In [16]:
print(data_dictionary)

{'10.0_10.0': 1, '10.0_11.0': 2, '10.0_12.0': 3, '10.0_13.0': 4, '10.0_14.0': 5, '10.0_15.0': 6, '10.0_16.0': 7, '10.0_17.0': 8, '10.0_19.0': 9, '10.0_20.0': 10, '10.0_21.0': 11, '10.0_22.0': 12, '10.0_23.0': 13, '10.0_24.0': 14, '10.0_25.0': 15, '10.0_5.0': 16, '10.0_6.0': 17, '10.0_7.0': 18, '10.0_78.0': 19, '10.0_8.0': 20, '10.0_9.0': 21, '15.0_10.0': 22, '15.0_11.0': 23, '15.0_12.0': 24, '15.0_13.0': 25, '15.0_14.0': 26, '15.0_15.0': 27, '15.0_16.0': 28, '15.0_17.0': 29, '15.0_18.0': 30, '15.0_19.0': 31, '15.0_20.0': 32, '15.0_21.0': 33, '15.0_22.0': 34, '15.0_23.0': 35, '15.0_24.0': 36, '15.0_25.0': 37, '15.0_5.0': 38, '15.0_6.0': 39, '15.0_7.0': 40, '15.0_8.0': 41, '15.0_9.0': 42, '5.0_10.0': 43, '5.0_11.0': 44, '5.0_12.0': 45, '5.0_13.0': 46, '5.0_14.0': 47, '5.0_15.0': 48, '5.0_16.0': 49, '5.0_17.0': 50, '5.0_18.0': 51, '5.0_19.0': 52, '5.0_20.0': 53, '5.0_21.0': 54, '5.0_22.0': 55, '5.0_23.0': 56, '5.0_24.0': 57, '5.0_25.0': 58, '5.0_5.0': 59, '5.0_6.0': 60, '5.0_7.0': 61, '5.

In [17]:
def encode(number):
    vector = -np.ones(len(data_dictionary))
    vector[number-1] = 1
    return vector

In [18]:
def decode(vector):
    return np.argmax(vector) + 1

In [19]:
x = encode(2)
print(x)
print(decode(x))

[-1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1.]
2


Pobranie danych z pliku:

In [20]:
def control(reading):
    return min(reading, 50)

In [29]:
def get_data(measurement):
    offset = 0
    sample_data = []
    with open(directory + "/" + measurement + ".txt", newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            if len(row) == 6 + offset:
                odczyt1 = float(row[1+offset])
                odczyt2 = float(row[3+offset])
                odczyt3 = float(row[5+offset])
                sample_data.append([np.array([control(odczyt1), control(odczyt2), control(odczyt3)]),
                                    encode(data_dictionary[measurement])])
    return sample_data

Rezultatem jest lista **data** składająca się z list w formie: \[ \[atrybuty\], \[macierz odpowiadająca gatunkowi\] \]

Podział danych na zbiór testowy według **train_test_ratio**:

In [30]:
def create_train_test_sets(train_test_ratio, data):
    np.random.shuffle(data)
    train_set_size = int(train_test_ratio*len(data))
    test_set_size = len(data) - train_set_size
    train_set = data[:train_set_size]
    test_set = data[-test_set_size:]
    return train_set, test_set

Podział danych w zbiorze ze względu na klasy:

In [31]:
def split_data(data):
    class_vector = [[] for element in data_dictionary]
    for row in data:
        class_vector[decode(row[1])-1].append(row)
    return class_vector

In [32]:
def make_3D_set(index, data):
    data_set = np.array([[row[0][0], row[0][1], row[0][2], row[1][index]] for row in data])
    return data_set

### Funkcje związane z SVM

Funkcja realizująca SVM **linear_SVM** przyjmuje jako argument zbiór z elementami postaci \[atrybuty, przynależność (1 lub -1) do danej klasy\] i zwraca **w** oraz **b** wyznaczające hiperpłaszczyznę dzielącą zbiór.  
Zmiennymi globalnymi są **TRAIN_SET**, **RECORD_SIZE** oraz **LAMBDA**, oznaczające odpowiednio wielkość podanego zbioru, ilość atrybutów dla elementu w zbiorze oraz $\lambda$. Zmienne te są globalne aby nie było potrzeby przekazywania ich jako parametry, do funkcji, która będzie minimalizowana (uznawała by je ona za zmienne i według nich szukała minimum).
Funkcja minimize zaimportowana z biblioteki SciPy wykorzystuje poniżej zdefiniowane funkcje **constraint** oraz **target**.

In [33]:
def linear_SVM(train_set):
    global TRAIN_SET
    global RECORD_SIZE
    global LAMBDA
    LAMBDA = 0.5
    TRAIN_SET = train_set
    RECORD_SIZE = len(TRAIN_SET[0]) - 1
    constraint = NonlinearConstraint(constraint1, np.zeros(len(TRAIN_SET)), np.inf)
    res = minimize(target, np.random.random(RECORD_SIZE + 1), constraints=constraint)
    w = res.x[:RECORD_SIZE]
    b = res.x[RECORD_SIZE]
    return w, b

Funkcja **target** jest równoważna:  

<center>$\sum_i\zeta_i + \lambda \|w\|$</center>
  
      
    
  
która jest minimalizowana dla argumentów *w* i *b* przez **minimize**. Argumenty te są zebrane w jednej liście aby funkcja optymalizująca mogła działać. 

In [34]:
def target(w):
    return np.linalg.norm(w[:RECORD_SIZE], 2)*LAMBDA + dzeta(w).sum()

Na minimalizację powyższej funkcji nałożone są warunki:  
<center>$y_i(w^Tx - b)\geq 1 - \zeta_i$ </center>  
Co jest równoważne:  
<center>$y_i(w^Tx - b) - 1 + \zeta_i\geq 0 $ </center>  
Operacja ta w funkcji constarint jest wykonywana od razu dla całej macierzy, gdzie $\zeta$ jest obliczane od razu jako wektor.

In [35]:
def constraint1(w):
    return ((np.matmul(TRAIN_SET[:, :RECORD_SIZE], w[:RECORD_SIZE]) - w[RECORD_SIZE]) * TRAIN_SET[:, RECORD_SIZE]) + \
            dzeta(w) - np.ones(len(TRAIN_SET))

Funkcja **dzeta** wykonuje operację:
<center>$\zeta_i = max(1 - f(x_i)y_i, 0)$</center>  
dla każdego elementu macierzy z danymi i zwraca wynik jako wektor.


In [36]:
def dzeta(w):
    return np.maximum(np.zeros(len(TRAIN_SET)), np.ones(len(TRAIN_SET)) - ((np.matmul(TRAIN_SET[:, :RECORD_SIZE], w[:RECORD_SIZE]) - w[RECORD_SIZE]) * TRAIN_SET[:, RECORD_SIZE]))

### Uczenie SVM - program

Dane są ładowane do listy data, następnie tworzony jest zbiór testowy, który zostaje podzielony na 3 grupy. Uczone są 3 SVM (porównują zbiory 1 z 2, 2 z 3 i 3 z 1), które zwracają parametry *w* i *b* a ich wyniki są przekazywane do funkcji **predict**.

In [37]:
data = []
for element in data_dictionary:
    data += get_data(element)
print(len(data))
print(len(data_dictionary))
train_set, test_set = create_train_test_sets(0.9, data)
class_vector = split_data(train_set)
for element in class_vector:
    print(len(element))

63000
63
909
898
908
893
888
899
892
906
904
882
898
912
900
912
903
900
911
900
907
908
876
895
890
894
898
925
882
904
922
898
896
916
882
910
893
903
900
909
906
896
883
888
898
917
894
885
911
893
893
899
895
904
897
904
903
917
898
906
896
894
882
905
913


In [38]:
parameters_array = [[[] for _ in range (len(data_dictionary))] for _ in range (len(data_dictionary))] 
for i in range (len(data_dictionary)):
    for j in range (len(data_dictionary)):
        if i!=j:
            parameters_array[i][j] = linear_SVM(make_3D_set(i, class_vector[i] + class_vector[j]))

#### Funkcja predict

Funkcja **predict** przyjmuje jako argument atrybuty elementu zbioru i na ich podstawie decyduje do jakiej klasy należy dany element. Robi to instrukcjami warunkowymi (jeśli widzi, że dla porównań 1 z 2 i 1 z 3 oba wskazania są na 1 to zwraca macierz odpowiadającą elementowi klasy pierwszej, analogicznie dla pozostałych zbiorów). Sytuację, w której porówniane 1 z 2 wskazuje na 2, 2 z 3 na 2 i 3 z 1 na 1 pozostawiłem nierozstrzygnięte. Porównania między dwoma klasami odbywają się za pomocą obliczenia $wx - b$ i zwrócenia 1 jeśli wynik jest większy niż 0 lub -1 jeśli mniejszy.

Funkcja progowa unipolarna zwraca 1 jeśli *x* jest większy lub równy *a* i -1 jeśli *x* jest mniejszy od *a*  

In [39]:
def threshold_unipolar_function(x, a):
    if (x >= a):
        return 1
    elif (x < a):
        return -1

In [40]:
def predict(parameters_array, element):
    for i in range (len(data_dictionary)):
        pred_vector = [(threshold_unipolar_function(np.matmul(parameters_array[i][j][0], element) - parameters_array[i][j][1], 0)) 
                       for j in range(len(data_dictionary)) if i!=j]
#         print(pred_vector)
        if pred_vector == list(np.ones(len(data_dictionary)-1)):
            return i+1
    return 0

In [41]:
len(test_set)

6300

### Ewaluacja - sprawdzenie działania algorytmu

Poniżej znajduje się fragment programu, który sprawdza stopień poprawnośći przewidywań przez SVM. Zbiór testowy jest podzielony na 3 klasy i dla każdej klasy przewidywana jest jej klasa na podstawie modelu. Jeśli model się pomylił dodawana jest jedynka do liczby błędów.

In [42]:
data = []
for element in data_dictionary:
    data += get_data(element)
print(len(data))
print(len(data_dictionary))
train_set, test_set = create_train_test_sets(0.1, data)
class_vector = split_data(train_set)
for element in class_vector:
    print(len(element))

63000
63
111
94
93
111
99
103
91
70
122
99
108
102
82
94
100
112
98
107
86
103
97
99
109
88
103
92
97
93
88
104
97
102
82
94
96
100
100
99
96
108
104
119
112
101
96
85
110
87
95
88
110
98
113
101
109
108
105
98
103
104
111
99
115


In [31]:
parameters_array = [[[] for _ in range (len(data_dictionary))] for _ in range (len(data_dictionary))] 
for i in range (len(data_dictionary)):
    for j in range (len(data_dictionary)):
        if i!=j:
            parameters_array[i][j] = linear_SVM(make_3D_set(i, class_vector[i] + class_vector[j]))

In [43]:
classes_vector = split_data(test_set)
total_errors = 0
for element in data_dictionary.values():
    errors = 0
    for row in classes_vector[element-1]:
        if predict(parameters_array, row[0]) != element:
            errors += 1
    total_errors += errors
    print(f"Class {element}: accuracy {(1 - errors/len(classes_vector[element-1]))*100:.2f}%")
total_accuracy = (1 - (total_errors)/(len(test_set)))*100
print(f"Total accuracy {total_accuracy:.2f}%")

Class 1: accuracy 99.78%
Class 2: accuracy 99.78%
Class 3: accuracy 99.67%
Class 4: accuracy 99.55%
Class 5: accuracy 99.78%
Class 6: accuracy 99.67%
Class 7: accuracy 99.89%
Class 8: accuracy 88.92%
Class 9: accuracy 97.72%
Class 10: accuracy 98.67%
Class 11: accuracy 99.55%
Class 12: accuracy 99.89%
Class 13: accuracy 0.98%
Class 14: accuracy 0.66%
Class 15: accuracy 0.00%
Class 16: accuracy 0.00%
Class 17: accuracy 0.00%
Class 18: accuracy 95.86%
Class 19: accuracy 0.00%
Class 20: accuracy 99.89%
Class 21: accuracy 99.89%
Class 22: accuracy 100.00%
Class 23: accuracy 99.21%
Class 24: accuracy 5.15%
Class 25: accuracy 69.12%
Class 26: accuracy 98.68%
Class 27: accuracy 91.25%
Class 28: accuracy 10.92%
Class 29: accuracy 99.78%
Class 30: accuracy 60.60%
Class 31: accuracy 92.80%
Class 32: accuracy 100.00%
Class 33: accuracy 99.67%
Class 34: accuracy 100.00%
Class 35: accuracy 93.36%
Class 36: accuracy 58.00%
Class 37: accuracy 0.00%
Class 38: accuracy 0.00%
Class 39: accuracy 0.00%
Cl

In [33]:
"""
Sample test
"""
for _ in range(10):
    x = random.choice(data)
    print(decode(x[1]), predict(parameters_array, x[0]))
    


13 13
5 9
6 6
14 9
5 0
8 8
1 1
2 2
8 8
10 9
